<a href="https://colab.research.google.com/github/KonstantinKlepikov/3Dreconstruction/blob/master/Fashion_Lenet_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Проверка работы драйверов GPU.  
Чтобы всё работало, нужно в  Runtime->Change Runtime Type выбрать GPU

In [0]:
!nvidia-smi

Sat Mar 30 13:04:14 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

Подключение библиотек

In [0]:
import numpy as np
import cv2

import sys
import os

import keras

Using TensorFlow backend.


В keras довольно много своих модельных наборов данных.  
Мы возьмем FashionMNIST - набор данных от Zalando с 10 классами разного вида одежды.

In [0]:

from keras.datasets import fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()


4423680/4422102 [==============================] - 1s 0us/step


Импортируем всякое  - объект модели, объекты для слоёв, оптимизатор, коллбэки, утилиты.

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

from keras.optimizers import SGD

from keras.callbacks import TerminateOnNaN, ModelCheckpoint

from keras.utils.np_utils import to_categorical

from sklearn.model_selection import train_test_split

import tensorflow as tf


Функция для расписания скорости градиентного спуска по эпохам(понадобится на следующей неделе)

In [0]:
def step_decay(epoch):
    initial_lrate = 0.01
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * np.power(drop, np.floor((1+epoch)/epochs_drop))
    return lrate


Сам градиентный спуск:

In [0]:

opt = SGD(lr = 0.001)


Instructions for updating:
Colocations handled automatically by placer.


Описание модели:

In [0]:
model = Sequential()
#Layer 1
#Conv Layer 1
model.add(Conv2D(filters = 6, 
                 kernel_size = 5, 
                 strides = 1, 
                 activation = 'relu', 
                 input_shape = (32,32,1)))
#Pooling layer 1
model.add(MaxPooling2D(pool_size = 2, strides = 2))
#Layer 2
#Conv Layer 2
model.add(Conv2D(filters = 16, 
                 kernel_size = 5,
                 strides = 1,
                 activation = 'relu',
                 input_shape = (14,14,6)))
#Pooling Layer 2
model.add(MaxPooling2D(pool_size = 2, strides = 2))
#Flatten
model.add(Flatten())
#Layer 3
#Fully connected layer 1
model.add(Dense(units = 120, activation = 'relu'))
#Layer 4
#Fully connected layer 2
model.add(Dense(units = 84, activation = 'relu'))
#Layer 5
#Output Layer
model.add(Dense(units = 10, activation = 'softmax'))
model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])



Подготовка данных под модель.

In [0]:

x_train = x_train.reshape(60000, 28, 28,1)

x_test = x_test.reshape(10000, 28, 28, 1)

x_test_padded = []
for img in x_test:
    img = np.pad(img,((2,2),(2,2),(0,0)), 'constant')
    x_test_padded.append(img)

x_test_padded = np.stack(x_test_padded)

x_train_padded = []
for img in x_train:
    img = np.pad(img,((2,2),(2,2),(0,0)), 'constant')
    x_train_padded.append(img)

x_train_padded = np.stack(x_train_padded)

y_train = to_categorical(y_train)

y_test = to_categorical(y_test)


Объявление коллбэков:

In [0]:

callbacks = [
    TerminateOnNaN(), ModelCheckpoint('fashion_lenet.hdf5', verbose=1,monitor='val_loss', save_best_only=True),
   # LearningRateScheduler(step_decay)
]


Обучение:

In [0]:

history = model.fit(x_train_padded,y_train, steps_per_epoch=128, epochs = 50, shuffle=True, validation_data=(x_test_padded,y_test), validation_steps=32, callbacks=callbacks)


Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/50
128/128 [==============================] - 94s 734ms/step - loss: 5.6265 - acc: 0.4379 - val_loss: 1.0771 - val_acc: 0.6543

Epoch 00001: val_loss improved from inf to 1.07706, saving model to fashion_lenet.hdf5
Epoch 2/50
 11/128 [=>............................] - ETA: 1:17 - loss: 1.0276 - acc: 0.6658

История обучения:

In [0]:

history.history.keys()

import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(history.history['acc'], c='blue')
plt.plot(history.history['val_acc'], c='green')

plt.plot(history.history['loss'][100:], c='blue')
plt.plot(history.history['val_loss'][100:], c='green')

